# Loading STL from the Web

In [ ]:
from ipywidgets import Text, HBox, Button, HTML, Dropdown
from k3d import K3D
from IPython.display import display
import requests

K3D supports the ASCII format for STL, but the binary format is more popular on sites like [Thingiverse](https://www.thingiverse.com/). [numpy-stl](https://github.com/WoLpH/numpy-stl) supports reading and writing both formats. You can install it with `pip`:

In [ ]:
!pip install numpy-stl

In [ ]:
from stl.stl import StlMesh, ASCII
from tempfile import NamedTemporaryFile

In [ ]:
plot = K3D()
stl = None

In addition to the plot, let's use some widgets and events to handle loading things from the web with `requests`.

In [ ]:
thing = Text(placeholder="URL of STL", value="http://www.thingiverse.com/download:99668")
btn = Button(icon="fa-cloud-download", button_style="primary", description=" Load")
status = HTML()

@btn.on_click
def load(_):
    global stl, plot
    btn.button_style = "warning"
    btn.icon = "fa-spinner fa-spin"
    btn.description = " Loading..."
    status.value = ""
    
    if stl is not None:
        plot - stl

    try:
        with NamedTemporaryFile(delete=True) as tmp:
            r = requests.get(thing.value, stream=True)
            [tmp.write(chunk) for chunk in r.iter_content(chunk_size=1024)]
            mesh = StlMesh(filename=tmp.name)
        with NamedTemporaryFile(delete=True) as tmp:
            mesh.save(tmp.name, mode=ASCII)
            stl = K3D.stl(tmp.read())
        plot += stl
        btn.icon = "fa-cloud-download"
        btn.button_style = "success"
        btn.description = " Loaded!"
    except Exception as err:
        btn.icon = "fa-exclamation-triangle"
        btn.button_style = "danger"
        btn.description = " Failed!"
        status.value = "<pre>{}</pre>".format(err)
        raise err

The K3D widget can't go inside the standard layout widgets, but by `display`ing two widgets in the same cell, you can emulate a VBox. Try pressing the Load button, or copy/pasting some other examples:
- [castanets](http://www.thingiverse.com/download:99668)
- [python logo](https://www.thingiverse.com/download:1420085)
- [klein bottle](http://www.thingiverse.com/download:862590)

In [ ]:
display(HBox(children=[thing, btn, status]))
display(plot)